# Importing the Required Libraries



In [54]:
!pip install --upgrade gspread gspread-dataframe google-auth

In [55]:
# Required Datasets
Sales_full = 'Sales_full'
Sales_mini = 'Sales_mini'
Current_price = 'Current_price'
COGs = 'COGs.xlsx'
FBA_US = 'US Fee Preview.csv'
FBA_EU ='EU Fee Preview.csv'
Removal_Fees = 'Removal Fees.xlsx'
Montly_Storage_Fees_US = 'US Monthly Storage Fees.csv'
Montly_Storage_Fees_EU = 'EU Monthly Storage Fees.csv'
MB_File = 'MB File.xlsx'


In [56]:
# Web requests for APIs or downloading files
import requests

# Math functions (sqrt, ceil, floor, trigonometry, etc.)
import math

# Data analysis and manipulation
import pandas as pd
import numpy as np

# File system operations
import os
import io
from os import listdir
from os.path import isfile, join

# Excel file handling
import openpyxl
from openpyxl import load_workbook

# Date and time utilities
import datetime
import calendar
from datetime import date, datetime
from dateutil.relativedelta import relativedelta  # For month/year offsets

# Google Colab integrations
from google.colab import drive  # Mount Google Drive
from google.colab import auth   # Authenticate Google account

# Google API client libraries
from googleapiclient.http import MediaIoBaseDownload  # For downloading files
from google.auth import default                       # Default authentication
from googleapiclient.discovery import build           # Build Google service APIs
from googleapiclient.errors import HttpError          # Handle API errors


In [57]:
# First of all, we need to make sure that my directory is in My Drive
drive.mount('/content/drive')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Sales Data/MB'

# Our Functions

### API and Google docs Access

In [58]:
def get_spreadsheet_id(url):

    """
    Extract the Google Spreadsheet ID from a given Google Sheets URL.

    Args:
        url (str): The full Google Sheets URL.
                   Example: https://docs.google.com/spreadsheets/d/<SPREADSHEET_ID>/edit#gid=0

    Returns:
        str: The extracted spreadsheet ID.
    """
    # Split the URL at '/d/' and take the second part,
    # then split again at '/' to isolate the spreadsheet ID
    return url.split("/d/")[1].split("/")[0]

In [59]:
def get_information_about_file_in_drive(file_id):

  """
    Retrieve and display metadata about a file stored in Google Drive.

    Args:
        file_id (str): The Google Drive file ID or a Google Sheets URL.
                       If a URL is passed, the function will extract the ID.

    Returns:
        None: Prints file details (name, ID, owner, and type) or error messages if access fails.
  """

  # Authenticate the user in Google Colab
  auth.authenticate_user()

  # Get default credentials and build the Drive API service
  creds, _ = default()
  drive_service = build('drive', 'v3', credentials=creds)
  # If the provided value is a full URL, extract the file ID
  file_id = get_spreadsheet_id(url) if "http" in file_id else file_id

  # Try to access the file metadata
  try:
      file = drive_service.files().get(fileId=file_id, fields="id, name, mimeType, owners").execute()

      # Print file details
      print(f"✅ You have access to the file: {file['name']}")
      print(f"📄 File ID: {file['id']}")
      print(f"👤 Owner: {file['owners'][0]['emailAddress']}")
      print(f"📁 Type: {file['mimeType']}")

  # Handle common errors from the API
  except HttpError as e:
      if e.resp.status == 404:
            print("❌ File not found.")
      elif e.resp.status == 403:
          print("❌ You do NOT have access to this file.")
      else:
        print(f"⚠️ Error occurred: {e}")

In [60]:
def get_sheet_names(file_id):

  """
    Retrieve and display all sheet names from an Excel file.

    Args:
        file_id (str): Path to the Excel file or file-like object.

    Returns:
        list: A list of sheet names contained in the Excel file.
  """

  # Load the Excel file into a Pandas ExcelFile object
  xlsx = pd.ExcelFile(file_id)

  # Extract all sheet names from the Excel file
  sheet_names = xlsx.sheet_names

  # Print the sheet names for quick reference
  print(sheet_names)

### Currency Exchange API

In [61]:
def get_exchange_rate(base_currency, target_currency):

    """
    Fetches the latest exchange rate from `base_currency` to `target_currency`.

    Parameters:
        base_currency (str): The currency code to convert from (e.g., "USD").
        target_currency (str): The currency code to convert to (e.g., "EUR").

    Returns:
        float or None: The exchange rate if available, otherwise None.

    Example:
        rate = get_exchange_rate("USD", "EUR")
        if rate:
            print(f"1 USD = {rate} EUR")
        else:
            print("Rate not found.")
    """

    url = f"https://open.er-api.com/v6/latest/{base_currency}"

    ## Connect with the API
    response = requests.get(url)
    data = response.json()

    # Get the target rate
    if response.status_code == 200 and "rates" in data:
        rate = data["rates"].get(target_currency)
        return rate
    else:
        return None

In [62]:
def Apply_Exchange_Rate(df, columns_to_convert, get_exchange_rate_func):

    """

    Converts values in specified columns to match 'System_currency' using exchange rates,
    while keeping all rows — converted or not.

    Parameters:
    - df: pandas DataFrame with 'System_currency' and 'Currency' columns.
    - columns_to_convert: list of column names to convert.
    - get_exchange_rate_func: function that takes (base_currency, target_currency) and returns rate.

    Returns:
    - df_converted: DataFrame with converted rows.

    """
    df_result = df.copy()
    exchange_rate_cache = {}

    # Identify rows that need conversion
    mask_diff = df_result['System_currency'] != df_result['Currency']
    df_diff = df_result[mask_diff]

    for (sc, oc) in df_diff[['System_currency', 'Currency']].drop_duplicates().itertuples(index=False):
        # Cache exchange rate
        if (sc, oc) not in exchange_rate_cache:
            rate = get_exchange_rate_func(sc, oc)
            exchange_rate_cache[(sc, oc)] = rate
        else:
            rate = exchange_rate_cache[(sc, oc)]

        if rate:
            mask = (df_result['System_currency'] == sc) & (df_result['Currency'] == oc)
            df_result.loc[mask, columns_to_convert] = df_result.loc[mask, columns_to_convert].astype(float, errors='ignore') * rate
            print(f"✔️ 1 {oc} = {rate} {sc}")
        else:
            print(f"❌ Failed to retrieve exchange rate for {oc} to {sc}")

    return df_result


### Date Manipulation

In [63]:
def Get_Previous_Months_With_Year(num_of_previous_months = 3):

    """
    Generate a dictionary of previous months with their corresponding years.

    Args:
        num_of_previous_months (int, optional): Number of past months to retrieve.
                                                Defaults to 3.

    Returns:
        dict: A dictionary where keys are month names (e.g., "July")
              and values are their corresponding years (e.g., 2025).

    Example:
        >>> Get_Previous_Months_With_Year(3)
        {'July': 2025, 'June': 2025, 'May': 2025}
    """
    # Get the current date and time
    now = datetime.now()

    # Initialize an empty dictionary to store months and years
    months = {}

    # Loop through the required number of previous months
    for i in range(1,num_of_previous_months+1):

        # Calculate the past date by subtracting months
        past_date = now - relativedelta(months=i)

        # Extract the month name (e.g., "July") and year (e.g., 2025)
        month_name = past_date.strftime("%B")
        year = past_date.year

        # Add the month-year pair to the dictionary
        months[month_name] = year

    return months

In [64]:
def filter_df_by_previous_months(df, num_of_previous_months = 3):

    """
    Filters a DataFrame to include only rows corresponding to the previous N months.

    This function uses `Get_Previous_Months_With_Year` to determine the relevant months and years,
    then filters the DataFrame based on 'Month' and 'Year' columns.

    Args:
        df (pd.DataFrame): The input DataFrame containing at least 'Month' and 'Year' columns.
        num_of_previous_months (int, optional): Number of previous months to include. Defaults to 3.

    Returns:
        pd.DataFrame: A DataFrame containing only the rows from the specified previous months.

    Example:
        filtered_df = filter_df_by_previous_months(df, num_of_previous_months=2)
    """
    dict_month_year = Get_Previous_Months_With_Year(num_of_previous_months)
    df_previous_months = pd.DataFrame()

    for month, year in dict_month_year.items():
        df_t = df[(df['Month'] == month) & (df['Year'] == year)]
        df_previous_months = pd.concat([df_previous_months, df_t], ignore_index=True)

    return df_previous_months


### Tab Generation

In [65]:
def generate_ads_spend_tab(df):

    """
    Generates a pivot table showing 'Ads spend' and 'Units Sold' for the previous three months,
    sorted chronologically by month.

    Args:
        df_full (pd.DataFrame): The original dataset containing at least:
            - 'Month' (e.g., "June")
            - 'Year' (e.g., 2025)
            - 'Marketplace', 'ASIN', 'Ads spend', 'Units Sold'

    Returns:
        pd.DataFrame: A pivot table with 'Marketplace' and 'ASIN' as index,
        columns as MultiIndex (metric -> month), filled with sums.
    """
    metric_order = ['Units Sold', 'Ads spend']

    # Step 1: Filter data for the last 3 months
    df_tabs = filter_df_by_previous_months(df)

    # Step 2: Create pivot table
    pivot = pd.pivot_table(
        df_tabs,
        index=['Marketplace', 'ASIN'],
        columns='Month',
        values=['Units Sold', 'Ads spend'],
        aggfunc='sum'
    )

    # Step 3: Sort months chronologically
    month_order = list(calendar.month_name)[1:]  # ['January', ..., 'December']
    pivot = pivot.reindex(
        columns=sorted(
            pivot.columns,
            key=lambda x: (month_order.index(x[1]), metric_order.index(x[0]))
            )
        )

    # Step 4: Final touches
    pivot.reset_index(inplace=True)
    pivot_filled = pivot.fillna(0)


    # Step 5: Calculate total Ads Spend and Units Sold over 3 months
    unit_cols = [col for col in pivot_filled.columns if col[0] == 'Units Sold']
    ad_cols = [col for col in pivot_filled.columns if col[0] == 'Ads spend']

    pivot_filled['Total Units Sold'] = pivot_filled[unit_cols].sum(axis=1)
    pivot_filled['Total Ads Spend'] = pivot_filled[ad_cols].sum(axis=1)


    # Step 6: Compute cost per unit sold
    pivot_filled['Cost per Unit Sold'] = (pivot_filled['Total Ads Spend'] * -1) / pivot_filled['Total Units Sold']
    pivot_filled['Cost per Unit Sold'] = pivot_filled['Cost per Unit Sold'].replace([float('inf'), -float('inf')], 0).fillna(0)
    pivot_filled['amazon-store'] = pivot_filled['Marketplace'].map(dict_maketplace_country)
    pivot_filled['Country'] = pivot_filled['Marketplace'].map(COGS_country_dict)


    return pivot_filled


In [66]:
def generate_refunds_tab(
    df_full,
    price_list,
    df_cogs,
    df_fba,
    df_removal_fees,
    dict_maketplace_country,
    COGS_country_dict,
    num_of_previous_months=6
):

    """
    Generates the refunds tab in the MB report.

    This function aggregates refund and sales data over the past N months (default 6 months), combines it
    with pricing, COGS, FBA fees, and removal fees, and calculates refund-related metrics
    such as refund amount, cost of returned items, and refund cost per unit.

    Steps performed:
        1. Filters `df_full` for the previous `num_of_previous_months` and calculates the
           average refund rate per ASIN and marketplace.
        2. Merges pricing information and calculates the total refund amount.
        3. Merges COGS data to compute total cost of goods for refunded items.
        4. Incorporates FBA fees and estimates return fees.
        5. Calculates removal fees based on weight brackets.
        6. Performs final calculations including cost of returned items, return percentage,
           and refund cost per unit.

    Args:
        df_full (pd.DataFrame): Complete sales dataset including 'Marketplace', 'ASIN',
                                'Units Sold', and 'Refunds'.
        price_list (pd.DataFrame): Price list with columns ['ASIN', 'Marketplace', 'Price'].
        df_cogs (pd.DataFrame): COGS data including ['ASIN', 'SKU', 'Country', 'Average Acquisition Price'].
        df_fba (pd.DataFrame): FBA fees data including ['ASIN','SKU','amazon-store','FBA Fees','item-package-weight','unit-of-weight'].
        df_removal_fees (pd.DataFrame): Removal fees by country and weight brackets including ['Country', 'From', 'To', 'Removal Fees', 'Percentage'].
        dict_maketplace_country (dict): Mapping of marketplace codes to Amazon store countries.
        COGS_country_dict (dict): Mapping of marketplace name to country domains.
        num_of_previous_months (int, optional): Number of past months to consider for refund aggregation. Defaults to 6.

    Returns:
        pd.DataFrame: A DataFrame with aggregated refund metrics including:
            - Units Sold
            - Refunds
            - Average Refund Rate last N months
            - Refund Amount
            - Total COGS
            - Return Fee
            - Removal Fee
            - Cost of Returned Items
            - Return Percentage per Item
            - Refund Cost per Unit
            - Country and amazon-store mappings

    Example:
        refund_tab = generate_refunds_tab(
            df_full=sales_data,
            price_list=prices,
            df_cogs=cogs_data,
            df_fba=fba_fees,
            df_removal_fees=removal_fees,
            dict_maketplace_country=marketplace_map,
            COGS_country_dict=cogs_country_map,
            num_of_previous_months=6
        )
    """

    # Step 1: Aggregate refund data for the last 6 months
    pivot = pd.pivot_table(
        filter_df_by_previous_months(df_full, num_of_previous_months),
        index=['Marketplace', 'ASIN'],
        values=['Units Sold', 'Refunds'],
        aggfunc='sum'
    ).reset_index()

    pivot = pivot[['Marketplace', 'ASIN', 'Units Sold', 'Refunds']]
    pivot['Average Refund Rate last 6 months'] = pivot['Refunds'] / pivot['Units Sold']

    # Step 2: Add pricing info
    pivot = pivot.merge(price_list[['ASIN', 'Marketplace', 'Price']], how='left', on=['ASIN', 'Marketplace'])
    pivot['Refund Amount'] = pivot['Refunds'] * pivot['Price']

    # Step 3: Add COGS info
    pivot['Country'] = pivot['Marketplace'].map(COGS_country_dict)
    pivot = pivot.merge(df_cogs[['ASIN', 'SKU' ,'Country', 'Average Acquisition Price']], how='left', on=['ASIN', 'Country']).fillna(0)
    pivot.rename(columns={'Average Acquisition Price': 'COGs'}, inplace=True)
    pivot['Total COGS'] = pivot['COGs'] * pivot['Refunds']

    # Step 4: Add FBA Fees
    pivot['amazon-store'] = pivot['Marketplace'].map(dict_maketplace_country)
    pivot = pivot.merge(
        df_fba[['ASIN','SKU' ,'amazon-store', 'FBA Fees', 'item-package-weight', 'unit-of-weight']],
        how='left',
        on=['ASIN', 'amazon-store']
    )
    pivot['Return Fee'] = (pivot['Refunds'] * pivot['FBA Fees']) * 0.5
    pivot.fillna(0, inplace=True)
    pivot.replace({'UK': 'GB'}, inplace=True)

    # Step 5: Removal Fees based on weight brackets
    df_merged = pivot.merge(df_removal_fees, on='Country', how='left')
    mask = (df_merged['item-package-weight'] > df_merged['From']) & \
           (df_merged['item-package-weight'] <= df_merged['To'])
    df_valid = df_merged[mask].copy()
    df_valid['Extra Fee'] = (df_valid['item-package-weight'] - df_valid['From']) * df_valid['Percentage']
    df_valid['Total Removal Fees'] = df_valid['Removal Fees'] + df_valid['Extra Fee']

    # Drop duplicates (based on original pivot keys to avoid many-to-many merges)
    df_final = df_valid.drop_duplicates(subset=['ASIN', 'Marketplace', 'item-package-weight', 'Country'])

    # Merge removal fees back into pivot
    pivot = pivot.merge(
        df_final[['Country', 'item-package-weight', 'Total Removal Fees']],
        on=['Country', 'item-package-weight'],
        how='left'
    )
    pivot.rename(columns={'Total Removal Fees': 'Removal Fees'}, inplace=True)

    # Step 6: Final refund-related calculations
    pivot['Removal Fee'] = pivot['Removal Fees'] * pivot['Refunds']
    pivot['Cost of Returned Items'] = pivot['Total COGS'] + pivot['Return Fee'] + pivot['Removal Fee']
    pivot['Return Percentage per Item'] = pivot['Cost of Returned Items'] / pivot['Refund Amount']
    pivot['Refund Cost per unit'] = pivot['Cost of Returned Items'] / pivot['Units Sold']
    pivot.replace([np.inf, -np.inf], 0, inplace=True)
    pivot['Country'] = pivot['Marketplace'].map(COGS_country_dict)
    pivot['amazon-store'] = pivot['Marketplace'].map(dict_maketplace_country)

    return pivot


In [67]:
def calculate_profitability(df, input_EU_all, referral_fee, vat, Digital_Services_Fee):
    if input_EU_all == 'N':
      country_col = 'Country'
    else:
      country_col = 'amazon-store'


    # Referral fee calculations
    df['Referral_limit'] = df['Country'].map(referral_fee)
    df['Referral Fee %'] = (df['Price'] > df['Referral_limit']).replace({True: 0.15, False: 0.10})
    df['Referral Fee in Amazon'] = df['Referral Fee %'] * df['Price']

    # VAT calculations
    df['vat percentage'] = df[country_col].map(vat)
    df['vat'] = (df['Price'] - (df['Price'] / (df['vat percentage'] + 1)))

    # Digital Services Fee calculations
    df['Digital_Services_Fee percentage'] = df[country_col].map(Digital_Services_Fee)
    df['Digital_Services_Fee'] = df.apply(
        lambda row: [
            row['Digital_Services_Fee percentage'][0] * row["Referral Fee in Amazon"],
            row['Digital_Services_Fee percentage'][1] * row["FBA Fees"]
        ],
        axis=1
    )
    df['Digital_Services_Fee_Sum'] = df['Digital_Services_Fee'].apply(sum)

    # Net Profit and Margin before Ads
    df['Net Profit before Ads'] = (
        df['Price']
        - (
            df['FBA Fees']
            + df['monthly_storage_cost_per_unit']
            + df['Referral Fee in Amazon']
            + df['vat']
            + df['Digital_Services_Fee_Sum']
            + df['Average Acquisition Price']
        )
    )
    df['Net Margin before Ads'] = df['Net Profit before Ads'] / df['Price']

    # Net Profit and Margin after Ads
    df['Net Profit after Ads'] = (
        df['Net Profit before Ads']
        - (df['Refund Cost per unit'] + df['Cost_per_Unit_Sold'])
    )
    df['Net Margin after Ads'] = df['Net Profit after Ads'] / df['Price']

    # Net Profit and Margin after Ads without COGS
    df['Net Profit after Ads without COGS'] = (
        df['Net Profit after Ads'] + df['Average Acquisition Price']
    )
    df['Net Margin after Ads without COGS'] = (
        df['Net Profit after Ads without COGS'] / df['Price']
    )

    if input_EU_all == 'N':
      add_col = None
    else:
      add_col = 'amazon-store'



    # Select columns for final output
    cols = [
        'PRODUCT', 'SKU', 'ASIN', 'Country', add_col,
        'Price', 'Average Production Price Per Unit', 'Average Shipping Price per Unit',
        'Average Acquisition Price', 'Referral Fee in Amazon', 'Referral Fee %',
        'Digital_Services_Fee_Sum', 'FBA Fees', 'monthly_storage_cost_per_unit',
        'vat', 'vat percentage', 'Net Profit before Ads', 'Net Margin before Ads',
        'Cost_per_Unit_Sold', 'Refund Cost per unit', 'Net Profit after Ads',
        'Net Margin after Ads', 'Net Profit after Ads without COGS',
        'Net Margin after Ads without COGS'
    ]

    # Remove None safely
    cols = [c for c in cols if c is not None]

    return df[cols]


### Apperance Functions

In [68]:
def ads_tab_agg_apperance(df):
    """
    Aggregates Ads performance data at the Country level and computes totals across months.

    Args:
        df (pd.DataFrame): Input data with ads tab data
    Returns:
        pd.DataFrame: Aggregated DataFrame grouped by PRODUCT, SKU, ASIN, Country
            with monthly totals and overall totals:
            - Total Units Sold
            - Total Ads Spend
    """

    original_cols = [col for col in df.columns if col not in ['Marketplace', 'amazon-store']]

    # Identify all unit and ad spend columns
    unit_cols = [col for col in df.columns if "Units_Sold" in col]
    ad_cols = [col for col in df.columns if "Ads_spend" in col]

    # Group by PRODUCT, SKU, ASIN, Country and sum numeric values
    grouped = df.groupby(['PRODUCT', 'SKU', 'ASIN', 'Country'], as_index=False)[unit_cols + ad_cols].sum()

    # Reculcate baesed on the region now
    grouped['Total_Units_Sold'] = grouped[unit_cols].sum(axis=1)
    grouped["Total_Ads_Spend"] = grouped[ad_cols].sum(axis=1)
    grouped["Cost_per_Unit_Sold"] = grouped["Total_Ads_Spend"] /  grouped['Total_Units_Sold']
    return grouped[original_cols]


In [69]:
def Refund_agg_apperance(df_merged):

  """
    Aggregates refund and sales data by product, SKU, ASIN, and country,
    and calculates refund-related metrics for performance analysis.

    The function performs the following operations:
        1. Aggregates key columns ('Units Sold', 'Refunds', 'Price', 'COGs', 'FBA Fees', 'Removal Fees')
           using sum or mean as appropriate.
        2. Calculates additional refund metrics:
            - Average Refund Rate over the last 6 months
            - Refund Amount
            - Total COGS for refunded items
            - Return Fee
            - Removal Fee
            - Cost of Returned Items
            - Return Percentage per Item
            - Refund Cost per unit
        3. Fills missing values with 0 and orders columns for clarity.
        4. Merges the aggregated pivot table with `df_mb` to retain original dataset context.

    Args:
        df_merged (pd.DataFrame): Input DataFrame containing at least the following columns:
            'Country', 'PRODUCT', 'SKU', 'ASIN', 'Units Sold', 'Refunds',
            'Price', 'COGs', 'FBA Fees', 'Removal Fees'.

    Returns:
        pd.DataFrame: A DataFrame merged with the original dataset (`df_mb`) including:
            - Aggregated sales and refund metrics per product/SKU/ASIN/country.
            - Calculated refund-related financial metrics.

    Example:
        df_final = Refund_agg_apperance(df_merged)
  """

  # Aggregating key columns
  pivot = pd.pivot_table(
        df_merged[cols].drop_duplicates(),
        index=['Country', 'PRODUCT', 'SKU', 'ASIN'],
        values=['Units Sold', 'Refunds', 'Price', 'COGs', 'FBA Fees', 'Removal Fees'],
        aggfunc={
        'Units Sold': 'sum',
        'Refunds': 'sum',
        'Price': 'mean',
        'COGs': 'mean',
        'FBA Fees': 'mean',
        'Removal Fees': 'mean'
        }).reset_index()


  # Calculating the mertrics
  pivot['Average Refund Rate last 6 months'] = pivot['Refunds'] / pivot['Units Sold']
  pivot['Refund Amount'] = pivot['Refunds'] / pivot['Price']
  pivot['Total COGS'] = pivot['Refunds'] / pivot['COGs']
  pivot['Return Fee'] = pivot['Refunds'] / pivot['FBA Fees']
  pivot['Removal Fee'] = pivot['Refunds'] / pivot['Removal Fees']
  pivot['Cost of Returned Items'] = pivot['Total COGS'] + pivot['Return Fee'] + pivot['Removal Fee']
  pivot['Return Percentage per Item'] = pivot['Cost of Returned Items'] / pivot['Refund Amount']
  pivot['Refund Cost per unit'] = pivot['Cost of Returned Items'] / pivot['Units Sold']

  # Replacing NAN values with zeros to overcome errors
  pivot.fillna(0, inplace=True)

  # Ordering our pivot table
  pivot = pivot[['PRODUCT', 'SKU', 'ASIN', 'Country', 'Units Sold', 'Refunds', 'Average Refund Rate last 6 months',
       'Price', 'Refund Amount', 'COGs','Total COGS','FBA Fees', 'Return Fee','Removal Fees',
       'Removal Fee', 'Cost of Returned Items','Return Percentage per Item', 'Refund Cost per unit']]


  return df_mb.merge(pivot, on = ['PRODUCT', 'SKU', 'ASIN', 'Country'], how = 'left' )

# Key Dictionaries for Important Data Mapping







In [70]:
dict_maketplace_country = { 'Amazon.co.uk': "GB",
                           'Amazon.de':'DE',
                            'Amazon.fr': 'FR',
                            'Amazon.es': 'ES',
                            'Amazon.it': 'IT',
                            'Amazon.nl': 'NL',
                            'Amazon.com.be': 'BE',
                            'Amazon.ie': 'IE',
                            'Amazon.pl':'PL',
                            'Amazon.com.tr': 'TR',
                            'Amazon.se': 'SE',
                            'Amazon.com': 'US',
                            'Amazon.ca': 'CA',
                            'Amazon.com.mx': 'MX'}




dict_maketplace_currency = { 'Amazon.co.uk': "GBP",
                           'Amazon.de':'EUR',
                            'Amazon.fr': 'EUR',
                            'Amazon.es': 'EUR',
                            'Amazon.it': 'EUR',
                            'Amazon.nl': 'EUR',
                            'Amazon.com.be': 'EUR',
                            'Amazon.ie': 'EUR',
                            'Amazon.pl':'EUR',
                            'Amazon.com.tr': 'EUR',
                            'Amazon.se': 'EUR',
                            'Amazon.com': 'USD',
                            'Amazon.ca': 'CAD',
                            'Amazon.com.mx': 'MXN'}



dict_system_sales_currency = { 'Amazon.co.uk': "GBP",
                           'Amazon.de':'GBP',
                            'Amazon.fr': 'GBP',
                            'Amazon.es': 'GBP',
                            'Amazon.it': 'GBP',
                            'Amazon.nl': 'GBP',
                            'Amazon.com.be': 'GBP',
                            'Amazon.ie': 'GBP',
                            'Amazon.pl':'GBP',
                            'Amazon.com.tr': 'GBP',
                            'Amazon.se': 'GBP',
                            'Amazon.com': 'USD',
                            'Amazon.ca': 'USD',
                            'Amazon.com.mx': 'USD'}


COGS_country_dict = { 'Amazon.co.uk': "UK",
                           'Amazon.de':'EU',
                            'Amazon.fr': 'EU',
                            'Amazon.es': 'EU',
                            'Amazon.it': 'EU',
                            'Amazon.nl': 'EU',
                            'Amazon.com.be': 'EU',
                            'Amazon.ie': 'EU',
                            'Amazon.pl':'EU',
                            'Amazon.com.EU': 'EU',
                            'Amazon.se': 'EU',
                            'Amazon.com': 'US',
                            'Amazon.ca': 'CA',
                            'Amazon.com.mx': 'MX'}





COGS_amazon_store_dict = { "GB": "GB",
                           'DE':'EU',
                            'FR': 'EU',
                            'ES': 'EU',
                            'IT': 'EU',
                            'NL': 'EU',
                            'BE': 'EU',
                            'IE': 'EU',
                            'PL':'EU',
                            'TR': 'EU',
                            'SE': 'EU',
                            'US': 'US',
                            'CA': 'CA',
                            'MX': 'MX'}




amazon_store_COGS_dict = { "GB": "GB",
                           'EU':'DE',
                            'EU': 'FR',
                            'EU': 'ES',
                            'EU': 'IT',
                            'EU': 'NL',
                            'EU': 'BE',
                            'EU': 'IE',
                            'EU':'PL',
                            'EU': 'TR',
                            'EU': 'SE',
                            'US': 'US',
                            'CA': 'CA',
                            'MX': 'MX'}


In [71]:
USA_columns = [(                       ' ',                      'Products'),
            (                     'USA',                           'SKU'),
            (                     'USA',                          'ASIN'),
            (           'ON HAND STOCK',        'Average Sales (7 days)'),
            (           'ON HAND STOCK',       'Average Sales (30 days)'),
            (           'ON HAND STOCK',               'Units in Amazon'),
            (           'ON HAND STOCK',       'Transit to AMZ from 3PL'),
            (           'ON HAND STOCK',                  'Units in AWD'),
            (           'ON HAND STOCK',               'Stocks with 3PL'),
            (           'ON HAND STOCK',                   'TOTAL UNITS')]


CA_columns =  [(      'Unnamed: 0_level_0',                   'Products'),
            (                      'CA',                        'SKU'),
            (                      'CA',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]


UK_columns = [(      'Unnamed: 0_level_0',                   'Products'),
            (                      'UK',                        'SKU'),
            (                      'UK',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]


EU_columns = [(      'Unnamed: 0_level_0',                    'PRODUCT'),
            (                      'EU',                        'SKU'),
            (                      'EU',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]


JP_columns = [(      'Unnamed: 0_level_0',                    'PRODUCT'),
            (                   'JAPAN',                        'SKU'),
            (                   'JAPAN',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]



columns_names_usa = ['Products', 'SKU', 'ASIN', 'Average Sales (7 days)', 'Average Sales (30 days)', 'Units in Amazon', 'Transit to AMZ from 3PL',  'Units in AWD',  'Stocks with 3PL', 'TOTAL UNITS']
columns_names_other_countries = ['Products', 'SKU', 'ASIN', 'Average Sales (7 days)', 'Average Sales (30 days)', 'Units in Amazon', 'Transit to AMZ from 3PL', 'Stocks with 3PL', 'TOTAL UNITS']

In [72]:
referral_fee = { "GB": 10,
                'DE': 10,
                 'FR': 10,
                 'ES': 10,
                 'IT': 10,
                 'NL': 10,
                 'BE': 10,
                 'IE': 10,
                 'PL':10,
                 'TR': 10,
                 'SE': 105,
                 'US': 10,
                 'CA': 15,
                 'MX': 10000000000000,
                 "EU": 10}

vat = { "GB": 0.2,
        'DE': .19,
        'FR': 0.2,
        'ES': 0.21,
        'IT': .22,
        'NL': 0.21,
        'BE': 0.21,
        'IE': 0.2,
        'PL':.21,
        'TR': .21,
        'SE': .21,
        'US': 0,
        'CA': 0,
        'MX': 0,
        "EU": .2}

Digital_Services_Fee = { "GB": [0.02, 0.02],
        'DE': [0, 0],
        'FR': [0.02, 0],
        'ES': [0.02, 0],
        'IT': [0.02,0],
        'NL':[0,0],
        'BE': [0,0],
        'IE': [0,0],
        'PL':[0.02,0],
        'TR': [0.02,0],
        'SE': [0,0],
        'US': [0,0],
        'CA': [0,0],
        'MX': [0,0],
        "EU": [0.02,0]}



# Loading the Required Libraries


## Loading the full stock file data

>> The availability of the full stock data here supports you when performing removal calculations or when you have many ASINs and need to retrieve their stock data, as **this script serves multiple purposes**.

In [73]:
# -------------------------------
# Step 1. Get File Information
# -------------------------------

url = "https://docs.google.com/spreadsheets/d/1fGDIkxG_QDuQtZUKssmz399nWGntz56O/edit?usp=sharing&ouid=115826017476220798647&rtpof=true&sd=true"
get_information_about_file_in_drive(url)


# -------------------------------
# Step 2. Authenticate Google Drive API
# -------------------------------
auth.authenticate_user() # Google Colab authentication
creds, _ = default()  # Get default Colab credentials
drive_service = build('drive', 'v3', credentials=creds)


# -------------------------------
# Step 3. Extract File ID
# -------------------------------
file_id = get_spreadsheet_id(url)



# -------------------------------
# Step 4. Download Excel File
# -------------------------------
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)

done = False
while not done:
    status, done = downloader.next_chunk()
    print(f"🔄 Download progress: {int(status.progress() * 100)}%")

# Move to beginning of file for openpyxl
fh.seek(0)


# -------------------------------
# Step 5. Load Workbook
# -------------------------------
wb = openpyxl.load_workbook(fh, data_only=True)



# -------------------------------
# Step 6. Define Sheet Configurations
# Each country has:
# - list of columns to use
# - list of column names (for renaming)
# -------------------------------
df_fs = pd.DataFrame()
dict_of_countries = {
    'USA ': [USA_columns, columns_names_usa],
    'CA': [CA_columns, columns_names_other_countries],
    'UK ': [UK_columns, columns_names_other_countries],
    'EU ': [EU_columns, columns_names_other_countries],
    'JP': [JP_columns, columns_names_other_countries]
}


# -------------------------------
# Step 7. Iterate Sheets and Clean Data
# -------------------------------

for country, (columns_to_use, column_names) in dict_of_countries.items():
    ws = wb[country]

    # Read sheet rows (skip hidden rows, but keep header row)
    data = []
    for i, row in enumerate(ws.iter_rows(values_only=True), start=1):
        if i == 1 or not ws.row_dimensions[i].hidden:
            data.append(row)

    # Convert to DataFrame
    df_s = pd.DataFrame(data)

    # Remove non-standard rows and extra columns
    df_s  = df_s.iloc[3:, 0:len(pd.read_excel(fh, header=[0, 1 ], sheet_name= country).columns)] # remove unformal columns and rows
    df_s.columns = pd.read_excel(fh, header=[0, 1 ], sheet_name= country).columns # give the columns its original names
    df_s = df_s[columns_to_use]  # Keep only needed columns
    df_s.columns = column_names  # Rename columns
    df_s['Country'] = country    # Add country label
    df_s = df_s[df_s['ASIN'].isna() == False ] # keep only the filled columns

    # Append to final dataset
    df_fs = pd.concat([df_s, df_fs], ignore_index=True)




# -------------------------------
# Step 8. Final Cleanup
# -------------------------------

df_fs.fillna(0,inplace = True) # Replace NaN with 0
df_fs['Country'] = df_fs['Country'].str.strip().replace({'USA': 'US'})  # Normalize country codes
df_fs['ASIN'] = df_fs['ASIN'].str.strip()   # Clean ASIN values




# SKU and products name duplication cleaning
df_asin_sku = df_fs[['Products', 'ASIN', 'Country', 'SKU']].copy()

df_asin_sku_grouped = (
    df_asin_sku
    .groupby(["ASIN", "Country"])
    .agg({
        "SKU": lambda x: ", ".join(sorted(set(x.dropna().astype(str)))),
        "Products": lambda x: ", ".join(sorted(set(x.dropna().astype(str))))
    })
    .reset_index()
)



# exclude the out of stock products
df_fs = df_fs[df_fs['TOTAL UNITS'] > 0 ]

# Pivot for structured analysis (grouped by product, ASIN, and country)
df_fs = pd.pivot_table(df_fs, index = ['ASIN', 'Country'], values = ['Average Sales (7 days)',
       'Average Sales (30 days)', 'Units in Amazon', 'Transit to AMZ from 3PL',
       'Stocks with 3PL', 'TOTAL UNITS',  'Units in AWD'], aggfunc = 'sum').reset_index()

df_fs = df_fs.merge(df_asin_sku_grouped, how = 'left', on =["ASIN", "Country"])


✅ You have access to the file: Full stocks count.xlsx
📄 File ID: 1fGDIkxG_QDuQtZUKssmz399nWGntz56O
👤 Owner: annie@tilcotrading.com
📁 Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet


🔄 Download progress: 100%


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/tmp/ipython-input-3976809241.py:97: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_fs.fillna(0,inplace = True) # Replace NaN with 0


>> Now, the in-stock data is stored in the **df_fs** dataframe


## Loading the Sales data

### Loading all sales information except pricing

All sales information comes from Sellerboard. We have stored this data in two main dataframes: sales_full and sales_mini. For more details, please refer to the documentation or the **Collect and Reprocess** notebook.

In [74]:
# Define the root directory for the sales data and set it as the working directory
root_path = r'/content/drive/My Drive/Sales Data/Final Shape of Files'
os.chdir(root_path)

# Define column names for Sales Full (SF) and Sales Mini (SM) datasets
df_sf_cols = [
    'Date','Marketplace','ASIN','SKU','SalesOrganic',
    'SalesSponsoredProducts', 'SalesSponsoredDisplay',
    'UnitsOrganic', 'UnitsPPC',
    'FBALongTermStorageFee', 'FBAStorageFee', 'FBAPerUnitFulfillmentFee'
]

df_sm_cols = [
    'Date', 'Marketplace','ASIN','SKU',
    'Refunds','RefundCost', 'Cost of Goods','Ads spend'
]

# Load the datasets (Sales_full and Sales_mini)
df_sf = pd.read_csv(Sales_full)   # Full sales dataset
df_sm = pd.read_csv(Sales_mini)   # Mini sales dataset

# Replace missing values with 0 to avoid errors in calculations
df_sf.fillna(0, inplace=True)
df_sm.fillna(0, inplace=True)

# Convert 'Date' column in Sales Mini dataset to datetime and extract year & month name
df_sm['Date'] = pd.to_datetime(df_sm['Date'], format="%d/%m/%Y")
df_sm['Year'] = df_sm['Date'].dt.year
df_sm['Month'] = df_sm['Date'].dt.month_name()

# Convert 'Date' column in Sales Full dataset to datetime and extract year & month name
df_sf['Date'] = pd.to_datetime(df_sf['Date'], format="%d/%m/%Y")
df_sf['Year'] = df_sf['Date'].dt.year
df_sf['Month'] = df_sf['Date'].dt.month_name()

# Calculate key metrics for the Sales Full dataset
df_sf['Gross Revenue'] = df_sf['SalesOrganic'] + df_sf['SalesSponsoredProducts'] + df_sf['SalesSponsoredDisplay']
df_sf['Units Sold'] = df_sf['UnitsOrganic'] + df_sf['UnitsPPC']
df_sf['FBA storage fee'] = df_sf['FBALongTermStorageFee'] + df_sf['FBAStorageFee'] + df_sf['FBAPerUnitFulfillmentFee']


# Keep only selected columns for further analysis
df_sf = df_sf[['Date', 'Marketplace', 'ASIN','SKU',
               'Year', 'Gross Revenue', 'Units Sold', 'FBA storage fee']]

# Add mapping for country, marketplace currency, and system sales currency (from predefined dictionaries)
df_sm['Country'] = df_sm['Marketplace'].map(dict_maketplace_country)
df_sm['Currency'] = df_sm['Marketplace'].map(dict_maketplace_currency)
df_sm['System_currency'] = df_sm['Marketplace'].map(dict_system_sales_currency)

# Convert financial columns in Sales Mini to a unified system currency
columns_to_convert = ['RefundCost','Cost of Goods', 'Ads spend']
df_sm_converted = Apply_Exchange_Rate(df_sm, columns_to_convert, get_exchange_rate)

# Apply same mapping and conversions for Sales Full dataset
df_sf['Country'] = df_sf['Marketplace'].map(dict_maketplace_country)
df_sf['Currency'] = df_sf['Marketplace'].map(dict_maketplace_currency)
df_sf['System_currency'] = df_sf['Marketplace'].map(dict_system_sales_currency)

columns_to_convert = ['Gross Revenue','FBA storage fee']
df_sf_converted = Apply_Exchange_Rate(df_sf, columns_to_convert, get_exchange_rate)

# Merge the converted Sales Full and Sales Mini datasets on key columns
df_full = df_sf_converted.merge(
    df_sm_converted,
    how='left',
    on=['Date', 'Marketplace', 'ASIN', 'SKU', 'Year', 'Country', 'Currency', 'System_currency']
)

✔️ 1 EUR = 1.150734 GBP
❌ Failed to retrieve exchange rate for nan to nan
✔️ 1 CAD = 1.378433 USD
✔️ 1 MXN = 18.738882 USD
✔️ 1 EUR = 1.150734 GBP
❌ Failed to retrieve exchange rate for nan to nan
✔️ 1 CAD = 1.378433 USD
✔️ 1 MXN = 18.738882 USD


>> Now, the required sales information is consolidated into the **df_full** dataframe

### The pricing data

>> Our pricing data is sourced from the Keepa API. For more details, please refer to the documentation or the **Getting Prices notebook** to better understand the process.

In [75]:
# Load the current price data from a CSV file into a DataFrame
df_cp = pd.read_csv(Current_price)

# Sort the data by 'Date' in descending order
# (so the most recent records appear first)
df_cp.sort_values('Date', ascending=False, inplace=True)

# Create a price list by keeping only the latest entry
# For each unique combination of ASIN, Region, and Marketplace
# By doing that, we have the pricing for each product in each marketplace
price_list = df_cp.drop_duplicates(
    subset=['ASIN', 'Region', 'Marketplace'],
    keep='first'
)


>> Now, the pricing for each product in each marketplace is stored in the **price_list** dataframe.

## Loading the COGs data and Removal fees rules

### COGs data

>> The complete COGs data is a key component of the MB report. We have a prepared file that reflects currency changes, since the exchange rate is based on the **product’s purchase date** rather than **today’s rate**

In [76]:
# Define the root directory where the sales data files are stored
root_path = r'/content/drive/My Drive/Sales Data/MB'

# Change the working directory to the defined path
os.chdir(root_path)

# Load the Cost of Goods (COGs) data from an Excel file into a DataFrame
df_cogs = pd.read_excel(COGs)

# Merge with Cost of Goods (COGS) table on SKU for profitability analysis
df_cogs = df_full[['ASIN', 'SKU']].drop_duplicates().merge(
    df_cogs,
    how='left',
    on=['SKU']
)

>> Now, the COGs data is stored in the **df_cogs** dataframe

### Removal Fees Rules

>> The Removal Fees data contains the rules used to define the removal cost per unit.



In [77]:
# Load the Removal Fees data from an Excel file into a DataFrame
df_removal_fees = pd.read_excel(Removal_Fees)

>> Now, the Removal Fees rules stored in the **df_removal_fees** dataframe

## Amazon Seller Central Data

### Fee Preview Report Data

>> **The Fee Preview report from Amazon Seller Central** provides important details about our products, including the **FBA fees per product** and **the product measurements**, which are used to define the removal fees per unit

In [78]:
# Read with proper BOM handling and rename
fba_us_columns = [ '?"sku"', 'asin', 'amazon-store', 'item-package-weight', 'unit-of-weight', 'currency', 'product-size-tier',
                  'expected-fulfillment-fee-per-unit']

fba_eu_columns = ['ï»¿"sku"', 'asin','amazon-store',
       'item-package-weight', 'unit-of-weight',
       'currency',
       'expected-domestic-fulfilment-fee-per-unit',
       'product-size-weight-band']

# Read CSV with latin-1 to remove BOM characters
df_fba_us = pd.read_csv(FBA_US, encoding='latin-1', usecols = fba_us_columns)
df_fba_eu = pd.read_csv(FBA_EU, encoding='latin-1', usecols=fba_eu_columns )


# Rename columns to match
df_fba_eu.columns = ['SKU','ASIN', 'amazon-store', 'item-package-weight', 'unit-of-weight',
       'product-size-tier', 'currency', 'FBA Fees']

df_fba_us.columns = ['SKU','ASIN', 'amazon-store', 'item-package-weight',
       'unit-of-weight', 'product-size-tier', 'currency', 'FBA Fees']

# Combine into one dataframe
df_fba = pd.concat([df_fba_eu, df_fba_us])


>> Now, the Fee Preview data is stored in the **df_fba** dataframe.

### Monthly Storage Fees

>> The Monthly Storage Fees report is one of the few reports that provides the monthly storage cost per unit, and this information is essential for the MB report

In [79]:
# Define the relevant columns to extract from the Monthly Storage Fees reports
monthly_storage_fees_columns = [
    'asin', 'country_code', 'weight', 'weight_units',
    'item_volume', 'volume_units',
    'estimated_monthly_storage_fee', 'average_quantity_on_hand'
]

# Load the Monthly Storage Fees reports for US and EU marketplaces
df_msfus = pd.read_csv(Montly_Storage_Fees_US, usecols=monthly_storage_fees_columns)
df_msfeu = pd.read_csv(Montly_Storage_Fees_EU, usecols=monthly_storage_fees_columns)

# Concatenate the two reports into one unified DataFrame
df_montlystorage_fees = pd.concat([df_msfus, df_msfeu])

# Compute the monthly storage cost per unit
# (total storage fee divided by average quantity on hand)
df_montlystorage_fees['Monthly storage cost per unit'] = (
    df_montlystorage_fees['estimated_monthly_storage_fee'] / df_montlystorage_fees['average_quantity_on_hand']
)

# Rename columns to have a consistent, cleaner format
df_montlystorage_fees.columns = [
    'ASIN', 'amazon-store', 'weight', 'weight_units',
    'item_volume', 'volume_units',
    'average_quantity_on_hand', 'estimated_monthly_storage_fee',
    'Monthly storage cost per unit'
]

# Pivot the table to aggregate values by ASIN, store, and volume unit
df_montlystorage_fees = pd.pivot_table(
    df_montlystorage_fees,
    index=['ASIN', 'amazon-store', 'volume_units'],  # group by
    values=['weight_units', 'item_volume', 'average_quantity_on_hand', 'estimated_monthly_storage_fee'],
    aggfunc={
        'weight_units': 'first',          # keep the first weight unit (assuming consistency)
        'item_volume': 'mean',            # average item volume
        'average_quantity_on_hand': 'sum',# total quantity on hand
        'estimated_monthly_storage_fee': 'sum'  # total fees
    },
    fill_value=0
).reset_index()

# Recalculate the monthly storage cost per unit, rounded down to 2 decimal places
df_montlystorage_fees['monthly_storage_cost_per_unit'] = np.floor(
    (df_montlystorage_fees['estimated_monthly_storage_fee'] / df_montlystorage_fees['average_quantity_on_hand']) * 100
) / 100

# Replace exact zero values with "< 0.01" for reporting clarity
df_montlystorage_fees['monthly_storage_cost_per_unit'].replace({0: "< 0.01"}, inplace=True)

# Drop intermediate calculation columns no longer needed
df_montlystorage_fees.drop(columns=['estimated_monthly_storage_fee', 'average_quantity_on_hand'], inplace=True)

# Map marketplace codes to country codes using a predefined dictionary
df_montlystorage_fees['Country'] = df_montlystorage_fees['amazon-store'].map(COGS_amazon_store_dict)

# Ensure uniqueness and fix naming differences (UK → GB)
df_montlystorage_fees.drop_duplicates(inplace=True)
df_montlystorage_fees.replace({'UK': 'GB'}, inplace=True)

/tmp/ipython-input-2886082234.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_montlystorage_fees['monthly_storage_cost_per_unit'].replace({0: "< 0.01"}, inplace=True)


# **The Output Layer**

In [80]:

# Asking the user if they have already entered their data into the MB File
manual_or_file = input("Have you entered your data into the MB File? (Y/N): \n").strip().upper()
input_EU_all = input("Show EU datails? (Y/N): \notherwise it will show aggregation of EU  based on (DE, ES, FR, IT)\n \n ").strip().upper()


# Initialize an empty dictionary to store product details if entered manually
product_dic = {}

# Case 1: User already has entered the products data into the MB File
if manual_or_file == 'Y':
    # Load the MB Excel file into a DataFrame
    df_mb = pd.read_excel(MB_File)

    # Print all rows of the DataFrame for confirmation
    print(df_mb.head(df_mb.shape[0]))

    # Check if the 'Country' column exists and contains 'EU'
    if np.isin('EU', df_mb['Country'].values).any():
        country_col = 'Country'
    else:
        # If not, fall back to 'amazon-store' and rename the column for consistency
        country_col = 'amazon-store'
        df_mb.rename(columns={'Country': 'amazon-store'}, inplace=True)

# Case 2: User wants to input products manually
else:
    more_product = 'Y'

    # Loop until the user chooses not to enter more products
    while more_product == 'Y':
        # Show the list of available countries from sales data
        print('Our available countries:\n', df_sm['Country'].dropna().unique())

        # Ask user for product details
        country = input('Enter the product country: \n').strip().upper()

        # Decide which column to use for country identification
        if 'EU' in country:
            country_col = 'Country'
        else:
            country_col = 'amazon-store'

        # Collect product details
        product_name = input('Enter the product Name: \n').strip()
        asin = input('Enter the product ASIN: \n').strip()
        SKU = input('Enter the product SKU: \n').strip()

        # Save the details into the product dictionary
        product_dic[country] = [product_name, asin, SKU]

        # Ask if the user wants to add more products
        more_product = input("More Products? (Y/N): \n").strip().upper()

    # Convert the dictionary of products into a DataFrame
    rows = [
        {'Country': k, 'PRODUCT': v[0], 'ASIN': v[1], 'SKU': v[2]}
        for k, v in product_dic.items()
    ]
    df_mb = pd.DataFrame(rows)

    # Standardize the country column name to match the expected format
    df_mb.rename(columns={'Country': country_col}, inplace=True)


Have you entered your data into the MB File? (Y/N): 
y
Show EU datails? (Y/N): 
otherwise it will show aggregation of EU  based on (DE, ES, FR, IT)
 
 y
                         PRODUCT           SKU        ASIN Country
0        Oil Diffuser Black  V.2    TCNADB-002  B0BX8SK463      EU
1         Oil Diffuser White V.2  TCNADWEU-003  B0CG9T4QY7      EU
2                Hand Tester Kit     HEKUK-001  B0CX23X75J      EU
3                Sinus Rinse Kit    TCNI-UK001  B08B1GTHQR      EU
4         Insulin Cooler (small)      TCIC-001  B0B8T9FLL1      UK
5         Insulin Cooler (small)      TCIC-001  B0B8T9FLL1      JP
6  Plastic Pill Cutter - Haining     PMPPC-001  B0DM6RCQRS      JP


In [81]:
merge_cols = ['ASIN', country_col]

# Step 1: Generate pivot table
ad_spend_tab = generate_ads_spend_tab(df_full)

# Step 2: If columns are MultiIndex, swap and sort them
if isinstance(ad_spend_tab.columns, pd.MultiIndex):
    ad_spend_tab.columns = ad_spend_tab.columns.swaplevel(0, 1)

    # Step 3: Flatten all MultiIndex columns
    ad_spend_tab.columns = [
        f"{month}_{metric}".replace(' ', '_').lstrip('_') if isinstance(month, str) else metric
        for month, metric in ad_spend_tab.columns
    ]

    # Step 4: Sort columns so months are in calendar order
    month_order = list(calendar.month_name)[1:]  # ['January', 'February', ..., 'December']

    # Separate fixed columns (like 'Marketplace', 'ASIN', etc.) from dynamic ones
    fixed_cols = [col for col in ad_spend_tab.columns if not any(m in col for m in month_order)]
    month_metric_cols = [col for col in ad_spend_tab.columns if any(m in col for m in month_order)]

    # Sort the dynamic month-metric columns
    month_metric_cols_sorted = sorted(
        month_metric_cols,
        key=lambda x: month_order.index(x.split('_')[0]) if x.split('_')[0] in month_order else 99
    )

    # Combine columns back in order

    ad_spend_tab = ad_spend_tab[fixed_cols + month_metric_cols_sorted]

# Step 5: Merge
df_merged = df_mb.merge(ad_spend_tab, how='left', on=merge_cols)
final_cols = [
        'PRODUCT', 'SKU', 'ASIN', 'Country', 'Marketplace', 'amazon-store'
        ] + month_metric_cols + [ 'Total_Units_Sold', 'Total_Ads_Spend', 'Cost_per_Unit_Sold']

df_merged = df_merged[final_cols]

if country_col == 'Country':
  if input_EU_all == 'N':
    list_of_official_countries = ['US', 'CA', 'MX', 'GB', 'DE', 'ES', 'FR', 'IT', 'JP']
    df_merged = df_merged[df_merged['amazon-store'].isin(list_of_official_countries)]
    df_show = ads_tab_agg_apperance(df_merged)
    df_show = df_mb.merge(df_show, on =list(df_mb.columns), how = 'left' )
    cols_to_keep = df_show.columns[:4]
  else:
    df_show = df_merged
    cols_to_keep = df_show.columns[:6]
else:
  df_show = df_merged
  cols_to_keep = df_show.columns[:6]






root_path = r'/content/drive/My Drive/Sales Data/MB'
os.chdir(root_path)
df_show.to_excel('output.xlsx', sheet_name='ads tab')
df_profitability = df_show[list(cols_to_keep) + ["Cost_per_Unit_Sold"]]
df_show

,PRODUCT,SKU,ASIN,Country,Marketplace,amazon-store,June_Units_Sold,June_Ads_spend,July_Units_Sold,July_Ads_spend,August_Units_Sold,August_Ads_spend,Total_Units_Sold,Total_Ads_Spend,Cost_per_Unit_Sold
0,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.com.be,BE,2.0,-1.265807,4.0,-3.118489,2.0,-3.141504,8.0,-7.525800,0.940725
1,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.de,DE,3.0,-11.875575,10.0,-14.246087,6.0,-10.598260,19.0,-36.719922,1.932627
2,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.es,ES,1.0,-8.711056,1.0,-14.821454,1.0,-6.524662,3.0,-30.057172,10.019057
3,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.fr,FR,2.0,-4.775546,6.0,-4.694995,2.0,-9.896312,10.0,-19.366853,1.936685
4,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.ie,IE,1.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.000000,-0.000000
5,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.it,IT,2.0,-4.062091,10.0,-4.902127,6.0,-5.339406,18.0,-14.303624,0.794646
6,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.nl,NL,0.0,-2.140365,1.0,-2.082829,0.0,-0.529338,1.0,-4.752531,4.752531
7,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.pl,PL,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
8,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,Amazon.com.be,BE,0.0,-3.348636,0.0,-1.357866,0.0,-2.025292,0.0,-6.731794,0.000000
9,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,Amazon.de,DE,3.0,-2.013784,2.0,-2.094336,0.0,-2.922864,5.0,-7.030985,1.406197


In [82]:
cols = ['PRODUCT', 'SKU', 'ASIN', 'Country', 'Marketplace','amazon-store', 'Units Sold',
       'Refunds', 'Average Refund Rate last 6 months', 'Price',
       'Refund Amount',  'COGs', 'Total COGS',
       'FBA Fees', 'Return Fee',
       'Removal Fees', 'Removal Fee', 'Cost of Returned Items',
       'Return Percentage per Item', 'Refund Cost per unit']


df_refund_tab = generate_refunds_tab(
    df_full,
    price_list,
    df_cogs,
    df_fba,
    df_removal_fees,
    dict_maketplace_country,
    COGS_country_dict,
    num_of_previous_months=6
).drop_duplicates()


df_merged = df_mb.merge(df_refund_tab, how='left', on=merge_cols)
df_merged = df_merged[cols].drop_duplicates()
df_merged.fillna(0, inplace = True)

if country_col == 'Country':
  if input_EU_all == 'N':
     list_of_official_countries = ['US', 'CA', 'MX', 'GB', 'DE', 'ES', 'FR', 'IT']
     df_merged = df_merged[df_merged['amazon-store'].isin(list_of_official_countries)]
     df_show = Refund_agg_apperance(df_merged.drop_duplicates())
     df_show.fillna(0, inplace = True)
     limit = 4
     cols_to_keep = df_show.columns[:limit]
  else:
    df_show = df_merged
    limit = 6
    cols_to_keep = df_show.columns[:limit]
else:
  df_show = df_merged
  cols_to_keep = df_show.columns[:limit]



with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    df_show.to_excel(writer, sheet_name="refund Tab", index=False)

df_profitability = df_profitability.merge(df_show[list(cols_to_keep) + ['Price', 'FBA Fees', 'Refund Cost per unit']], on = list(df_show.columns[:limit]), how = 'left')
df_show

,PRODUCT,SKU,ASIN,Country,Marketplace,amazon-store,Units Sold,Refunds,Average Refund Rate last 6 months,Price,Refund Amount,COGs,Total COGS,FBA Fees,Return Fee,Removal Fees,Removal Fee,Cost of Returned Items,Return Percentage per Item,Refund Cost per unit
0,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.com.be,BE,14.0,1.0,0.071429,0.00,0.00,11.46,11.46,3.65,1.825,0.73,0.73,14.015,0.000000,1.001071
1,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.de,DE,37.0,3.0,0.081081,25.99,77.97,11.46,34.38,3.24,4.860,0.73,2.19,41.430,0.531358,1.119730
2,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.es,ES,5.0,0.0,0.000000,38.99,0.00,11.46,0.00,3.84,0.000,0.73,0.00,0.000,0.000000,0.000000
3,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.fr,FR,22.0,3.0,0.136364,29.99,89.97,11.46,34.38,5.17,7.755,0.73,2.19,44.325,0.492664,2.014773
4,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.ie,IE,1.0,0.0,0.000000,0.00,0.00,11.46,0.00,1.78,0.000,0.73,0.00,0.000,0.000000,0.000000
5,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.it,IT,30.0,5.0,0.166667,25.99,129.95,11.46,57.30,4.64,11.600,0.73,3.65,72.550,0.558292,2.418333
6,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.nl,NL,3.0,1.0,0.333333,0.00,0.00,11.46,11.46,3.61,1.805,0.73,0.73,13.995,0.000000,4.665000
7,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.pl,PL,0.0,0.0,0.000000,0.00,0.00,11.46,0.00,3.90,0.000,0.73,0.00,0.000,0.000000,0.000000
8,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,Amazon.se,SE,1.0,0.0,0.000000,0.00,0.00,11.46,0.00,0.00,0.000,0.00,0.00,0.000,0.000000,0.000000
9,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,Amazon.com.be,BE,0.0,0.0,0.000000,0.00,0.00,10.96,0.00,3.65,0.000,0.73,0.00,0.000,0.000000,0.000000


In [83]:
prof_cols = ['PRODUCT', 'SKU', 'ASIN', 'Country', 'amazon-store', 'monthly_storage_cost_per_unit',
             'Average Production Price Per Unit','Average Shipping Price per Unit', 'Average Acquisition Price']


df_merged = df_mb.replace({'UK': 'GB'}).merge(df_montlystorage_fees, how='left', on=merge_cols).drop_duplicates()
df_merged.merge(df_cogs.replace({'UK':'GB'}), how = 'left', on = ['ASIN', 'SKU', 'Country']).drop_duplicates()
df_show = df_merged.merge(df_cogs.replace({'UK':'GB'}), how = 'left', on = ['ASIN', 'SKU', 'Country']).dropna()

m_cols = list(df_profitability.columns[:limit])

if input_EU_all == 'N':
  list_of_official_countries = ['US', 'CA', 'MX', 'GB', 'DE', 'ES', 'FR', 'IT']
  df_show = df_show[df_show['amazon-store'].isin(list_of_official_countries)]
  #df_show.dropna(inplace = True)
  df_show.drop(columns = 'amazon-store', inplace = True)
  df_show = df_show.drop_duplicates()
  if 'amazon-store' in prof_cols:
    prof_cols.remove('amazon-store')

if 'Marketplace' in m_cols:
    m_cols.remove('Marketplace')


with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    df_show.to_excel(writer, sheet_name="COGs data", index=False)



df_profitability = df_profitability.replace({'UK':'GB'}).merge(df_show[prof_cols],  on = m_cols, how = 'left').dropna().reset_index(drop = True)

df_show

,PRODUCT,SKU,ASIN,Country,amazon-store,volume_units,item_volume,weight_units,monthly_storage_cost_per_unit,Product Name,Average Production Price Per Unit,Average Shipping Price per Unit,Average Acquisition Price
0,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,DE,cubic meters,0.0007,kilograms,0.02,Oil Diffuser Black V.2,10.75,0.71,11.46
1,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,ES,cubic meters,0.0007,kilograms,0.02,Oil Diffuser Black V.2,10.75,0.71,11.46
2,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,FR,cubic meters,0.0007,kilograms,0.02,Oil Diffuser Black V.2,10.75,0.71,11.46
3,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,IE,cubic meters,0.0007,kilograms,0.01,Oil Diffuser Black V.2,10.75,0.71,11.46
4,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,IT,cubic meters,0.0007,kilograms,0.02,Oil Diffuser Black V.2,10.75,0.71,11.46
5,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,SE,cubic meters,0.0007,kilograms,0.23,Oil Diffuser Black V.2,10.75,0.71,11.46
6,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,DE,cubic meters,0.0007,kilograms,0.02,Oil Diffuser White V.2,10.75,0.21,10.96
7,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,ES,cubic meters,0.0007,kilograms,0.02,Oil Diffuser White V.2,10.75,0.21,10.96
8,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,FR,cubic meters,0.0007,kilograms,0.02,Oil Diffuser White V.2,10.75,0.21,10.96
9,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,IE,cubic meters,0.0007,kilograms,0.01,Oil Diffuser White V.2,10.75,0.21,10.96


In [84]:
show_cols= ['PRODUCT', 'SKU', 'ASIN', country_col, 'Average Sales (7 days)',
       'Average Sales (30 days)', 'Units in Amazon', 'Units in AWD', 'Transit to AMZ from 3PL',
       'Stocks with 3PL', 'TOTAL UNITS']

merge_cols = ['ASIN', country_col]


if country_col == 'amazon-store':
  df_cogs_map = pd.DataFrame(list(COGS_amazon_store_dict.items()), columns=['amazon-store', 'Country'])
  df_fs_with_amazon_store = df_fs.merge(df_cogs_map, how='left', on='Country')
else:
  df_fs_with_amazon_store = df_fs.copy()


df_merged = pd.merge(
    df_mb,
    df_fs_with_amazon_store[['ASIN', 'Average Sales (7 days)', 'Average Sales (30 days)',
           'Units in Amazon', 'Transit to AMZ from 3PL',
           'Stocks with 3PL', 'TOTAL UNITS', country_col, 'Units in AWD']],
    on =merge_cols,
    how='left'
)


df_merged = df_merged[show_cols]
with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    df_merged.to_excel(writer, sheet_name="Stock data", index=False)
df_merged

,PRODUCT,SKU,ASIN,Country,Average Sales (7 days),Average Sales (30 days),Units in Amazon,Units in AWD,Transit to AMZ from 3PL,Stocks with 3PL,TOTAL UNITS
0,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,0.71,0.60,305.0,0.0,0.0,0.0,305.0
1,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,0.14,0.23,60.0,0.0,0.0,0.0,60.0
2,Hand Tester Kit,HEKUK-001,B0CX23X75J,EU,0.43,0.40,108.0,0.0,0.0,72.0,180.0
3,Sinus Rinse Kit,TCNI-UK001,B08B1GTHQR,EU,0.43,1.33,126.0,0.0,0.0,0.0,126.0
4,Insulin Cooler (small),TCIC-001,B0B8T9FLL1,UK,0.29,0.60,141.0,0.0,0.0,0.0,141.0
5,Insulin Cooler (small),TCIC-001,B0B8T9FLL1,JP,0.00,0.10,100.0,0.0,0.0,0.0,100.0
6,Plastic Pill Cutter - Haining,PMPPC-001,B0DM6RCQRS,JP,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df_fba['Country'] = df_fba['amazon-store'].map(COGS_amazon_store_dict)
df_fba['Country'].replace({"UK": "GB"})
merge_cols = ['ASIN', 'SKU', country_col]


with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    df_mb.merge(df_fba, on= merge_cols, how = 'left' ).drop_duplicates().to_excel(writer, sheet_name="Information about the products", index=False)
df_mb.merge(df_fba, on= merge_cols, how = 'left' ).drop_duplicates()

,PRODUCT,SKU,ASIN,Country,amazon-store,item-package-weight,unit-of-weight,product-size-tier,currency,FBA Fees
0,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,IE,320.01,grams,SmallParcel,EUR,1.78
1,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,PL,320.01,grams,SmallParcel,PLN,3.90
2,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,BE,320.01,grams,SmallParcel,EUR,3.65
3,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,NL,320.01,grams,SmallParcel,EUR,3.61
4,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,ES,320.01,grams,SmallParcel,EUR,3.84
5,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,IT,320.01,grams,SmallParcel,EUR,4.64
6,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,FR,320.01,grams,SmallParcel,EUR,5.17
7,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,DE,320.01,grams,SmallParcel,EUR,3.24
8,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,IE,329.99,grams,SmallParcel,EUR,1.78
9,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,PL,329.99,grams,SmallParcel,PLN,3.90


In [86]:
with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    calculate_profitability(df_profitability[df_profitability['Price']> 0 ],input_EU_all, referral_fee, vat, Digital_Services_Fee).to_excel(writer, sheet_name="profitability tab", index=False)

calculate_profitability(df_profitability[df_profitability['Price']> 0 ], input_EU_all, referral_fee, vat, Digital_Services_Fee)





/tmp/ipython-input-1059542741.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Referral_limit'] = df['Country'].map(referral_fee)
/tmp/ipython-input-1059542741.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Referral Fee %'] = (df['Price'] > df['Referral_limit']).replace({True: 0.15, False: 0.10})
/tmp/ipython-input-1059542741.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,PRODUCT,SKU,ASIN,Country,amazon-store,Price,Average Production Price Per Unit,Average Shipping Price per Unit,Average Acquisition Price,Referral Fee in Amazon,...,vat,vat percentage,Net Profit before Ads,Net Margin before Ads,Cost_per_Unit_Sold,Refund Cost per unit,Net Profit after Ads,Net Margin after Ads,Net Profit after Ads without COGS,Net Margin after Ads without COGS
0,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,DE,25.99,10.75,0.71,11.46,3.8985,...,4.149664,0.19,3.221836,0.123964,1.932627,1.119730,0.169479,0.006521,11.629479,0.44746
1,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,ES,38.99,10.75,0.71,11.46,5.8485,...,6.766860,0.21,10.93767,0.280525,10.019057,0.000000,0.918613,0.02356,12.378613,0.317482
2,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,FR,29.99,10.75,0.71,11.46,4.4985,...,4.998333,0.20,3.753197,0.125148,1.936685,2.014773,-0.198261,-0.006611,11.261739,0.375516
4,Oil Diffuser Black V.2,TCNADB-002,B0BX8SK463,EU,IT,25.99,10.75,0.71,11.46,3.8985,...,4.686721,0.22,1.206809,0.046434,0.794646,2.418333,-2.00617,-0.07719,9.45383,0.363749
5,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,DE,28.99,10.75,0.21,10.96,4.3485,...,4.628655,0.19,5.792845,0.199822,1.406197,2.662000,1.724648,0.059491,12.684648,0.437553
6,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,ES,35.99,10.75,0.21,10.96,5.3985,...,6.246198,0.21,9.417332,0.261665,0.762623,2.551875,6.102834,0.16957,17.062834,0.474099
7,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,FR,32.99,10.75,0.21,10.96,4.9485,...,5.498333,0.20,6.294197,0.190791,1.199065,1.679412,3.41572,0.103538,14.37572,0.43576
8,Oil Diffuser White V.2,TCNADWEU-003,B0CG9T4QY7,EU,IT,28.99,10.75,0.21,10.96,4.3485,...,5.227705,0.22,3.706825,0.127866,0.336590,0.000000,3.370235,0.116255,14.330235,0.494317
9,Hand Tester Kit,HEKUK-001,B0CX23X75J,EU,DE,27.99,8.48,1.49,9.97,4.1985,...,4.468992,0.19,5.782508,0.206592,13.595922,1.220455,-9.033868,-0.322753,0.936132,0.033445
10,Hand Tester Kit,HEKUK-001,B0CX23X75J,EU,ES,29.99,8.48,1.49,9.97,4.4985,...,5.204876,0.21,6.116654,0.203956,2.844614,2.490000,0.78204,0.026077,10.75204,0.358521
